In [1]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data")

In [3]:
df.columns = ["age", "workclass", "fnlwgt","education","education-num","marital-status","occupation","relationship","race",
              "sex","capital-gain","capital-loss","hours-per-week","native-country","income"]


In [4]:
pd.isnull(df).sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

In [5]:
df.isin([' ?']).sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
income               0
dtype: int64

In [6]:
df = df.drop(df[df["occupation"] == " ?"].index)
df = df.drop(df[df["workclass"] == " ?"].index)
df = df.drop(df[df["native-country"] == " ?"].index)
df.reset_index()
display(df)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32556,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32557,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32558,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [7]:
newdf = df

In [8]:
cleanup_workclass = {"workclass":     {" Private": 0, " Self-emp-not-inc":1, " Self-emp-inc":2, " Federal-gov":3, " Local-gov":4, " State-gov":5, " Without-pay":6, "Never-worked": 7},
                "sex": {" Male": 0, " Female": 1 },
                    "marital-status":{" Married-civ-spouse":0, " Divorced":1, " Never-married":2, " Separated":3, " Widowed":4, " Married-spouse-absent":5, " Married-AF-spouse":6},
                    "occupation":{" Tech-support":0, " Craft-repair":1, " Other-service":2, " Sales":3, " Exec-managerial":4, " Prof-specialty":5, " Handlers-cleaners":6, " Machine-op-inspct":7, " Adm-clerical":8, " Farming-fishing":9, " Transport-moving":10, " Priv-house-serv":11, " Protective-serv":12, " Armed-Forces":13},
                    "race":{" White":0, " Asian-Pac-Islander":1, " Amer-Indian-Eskimo":2, " Other":3, " Black":4},
                    "relationship":{" Wife":0, " Own-child":1, " Husband":2, " Not-in-family":3, " Other-relative":4, " Unmarried":5},
                    "native-country":{" United-States":0, " Cambodia":1, " England":2, " Puerto-Rico":3," Canada":4, " Germany":5, " Outlying-US(Guam-USVI-etc)":6, " India":7, " Japan":8, " Greece":9, " South":10, " China":11, " Cuba":12, " Iran":13, " Honduras":14, " Philippines":15, " Italy":16, " Poland":17, " Jamaica":18, " Vietnam":19, " Mexico":20, " Portugal":21, " Ireland":22, " France":23, " Dominican-Republic":24, " Laos":25, " Ecuador":26, " Taiwan":27, " Haiti":28, " Columbia":29, " Hungary":30, " Guatemala":31, " Nicaragua":32, " Scotland":33, " Thailand":34, " Yugoslavia":35, " El-Salvador":36, " Trinadad&Tobago":37, " Peru":38, " Hong":39, " Holand-Netherlands":40},
                    "income":{" <=50K":0, " >50K":1}}
newdf = newdf.replace(cleanup_workclass)
display(newdf)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,50,1,83311,Bachelors,13,0,4,2,0,0,0,0,13,0,0
1,38,0,215646,HS-grad,9,1,6,3,0,0,0,0,40,0,0
2,53,0,234721,11th,7,0,6,2,4,0,0,0,40,0,0
3,28,0,338409,Bachelors,13,0,5,0,4,1,0,0,40,12,0
4,37,0,284582,Masters,14,0,4,0,0,1,0,0,40,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,0,257302,Assoc-acdm,12,0,0,0,0,1,0,0,38,0,0
32556,40,0,154374,HS-grad,9,0,7,2,0,0,0,0,40,0,1
32557,58,0,151910,HS-grad,9,4,8,5,0,1,0,0,40,0,0
32558,22,0,201490,HS-grad,9,2,8,1,0,0,0,0,20,0,0


In [9]:
target_set = newdf["income"]

In [10]:
input_set = newdf.iloc[:,:-1]
input_set = input_set.drop('education', axis=1)

In [11]:
input_train, input_test, target_train, target_test = train_test_split(input_set, target_set, test_size=0.2, 
                                                                      random_state=42,shuffle=True)

In [16]:
from sklearn.model_selection import GridSearchCV
params = {'C': [10,100,200], 'kernel': ['rbf'], 'gamma': ['scale', 'auto']}

svm = SVC(random_state=42)
grid_search = GridSearchCV(svm, params, cv=5)
grid_search.fit(input_train, target_train)
print('Best hyperparameters:', grid_search.best_params_)
print('validation score:', grid_search.best_score_)

Best hyperparameters: {'C': 200, 'gamma': 'scale', 'kernel': 'rbf'}
validation score: 0.7928131515603092


In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
clf = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=200, probability=True))
clf.fit(input_train, target_train)
print(accuracy_score(clf.predict(input_test), target_test))

0.8402121664180342


In [18]:
from sklearn.metrics import confusion_matrix
confusion_matrix(target_train, clf.predict(input_train))

array([[17302,   816],
       [ 1829,  4181]], dtype=int64)

In [35]:
target_set = df["income"]
input_set = df.iloc[:,:-1]
input_train, input_test, target_train, target_test = train_test_split(input_set, target_set, test_size=0.2, random_state=42,shuffle=True)

In [105]:
from sklearn.model_selection import GridSearchCV
params = {'max_depth': [10, 12, 15 ], 'n_estimators': [200, 300,350]}
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, params, cv=5)
grid_search.fit(input_train, target_train)
print('Best hyperparameters:', grid_search.best_params_)
print('Validation score:', grid_search.best_score_)

Best hyperparameters: {'max_depth': 15, 'n_estimators': 300}
Validation score: 0.8630636813976109


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf_rf = RandomForestClassifier(n_estimators=300,max_depth=15, random_state=1,criterion = "gini")
clf_rf.fit(input_train, target_train)
print(accuracy_score(clf_rf.predict(input_test), target_test))

0.8586109729819327


In [15]:
from sklearn.metrics import confusion_matrix
confusion_matrix(target_train, clf_rf.predict(input_train))

array([[17530,   588],
       [ 1569,  4441]], dtype=int64)